In [1]:
from pycaret.regression import *

In [2]:
load_model?

Signature:
load_model(
    model_name,
    platform: Optional[str] = None,
    authentication: Optional[Dict[str, str]] = None,
    verbose: bool = True,
)
Docstring:
This function loads a previously saved pipeline.

Example
-------
>>> from pycaret.regression import load_model
>>> saved_lr = load_model('saved_lr_model')


model_name: str
    Name of the model.


platform: str, default = None
    Name of the cloud platform. Currently supported platforms:
    'aws', 'gcp' and 'azure'.


authentication: dict, default = None
    dictionary of applicable authentication tokens.

    when platform = 'aws':
    {'bucket' : 'Name of Bucket on S3', 'path': (optional) folder name under the bucket}

    when platform = 'gcp':
    {'project': 'gcp-project-name', 'bucket' : 'gcp-bucket-name'}

    when platform = 'azure':
    {'container': 'azure-container-name'}


verbose: bool, default = True
    Success message is not printed when verbose is set to False.


Returns:
    Trained Model
File:      